<a href="https://colab.research.google.com/github/samyzaf/notebooks/blob/main/bsm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div hidden>
$
\newcommand{\Dt}{\Delta t}
\newcommand{\Ds}{\Delta s}
\newcommand{\V}{\mathbf{V}}
$
</div>

# **Solving the Black-Scholes-Merton Formula by FDM**
<small>version: 26.12.2024a</small>

* **FDM** stands for **Finite Difference Method**.
* Thanks to
[Antoni Smolski](https://antonismolski.medium.com/?source=post_page-----965fd0539808--------------------------------)
for his inspiring BSM blog  
https://antonismolski.medium.com/implementation-of-black-scholes-formula-using-finite-difference-method-with-code-965fd0539808  
on which this notebook is based.

* Also many thanks to **QuantStart Newsletter** for a detailed
  outline of a **C++** code for an **Explicit Euler FDM** solution
  specific to the European call option equation at their blog  
  https://www.quantstart.com/articles/C-Explicit-Euler-Finite-Difference-Method-for-Black-Scholes/

* This notebook is part of a PDE course project at Israel
  Institute of Technology (Technion).
  Most of the theoretical background is explained
  in the project booklet (see the Moodle course page).

* The Python code in this notebook is essentially
  Antoni's Python code with
  needed modifications to suite our course needs and targets.

* The common **Black-Scholes-Merton equation** is the
  partial differential equation
  $$\large
  v_t +rsv_s + \frac{1}{2}\sigma^2 s^2 v_{ss} = r v
  $$
  which models the value of a financial derivative.

* For European Call Option, the equation is
  $$\large
  -v_t +rsv_s + \frac{1}{2}\sigma^2 s^2 v_{ss} = r v
  $$

* $v(s,t)$ is the option value of a given asset at
   price $s$,  at time $t$.

* $r$ = risk-free interest rate (aka **drift**)

* $\sigma$ = the price volatility measure of the stock
  (standard deviation of a
  [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution))

* We use Python's [Numpy package](https://numpy.org), to model
  the **FDM** grid structure which represents the discrete
  version of $v(s,t)$.

* We use our
  [**fdmtools Python package**](https://samyzaf.com/fdmtools.zip),
  which is installed and imported by the next code cell.
$
\newcommand{\Dt}{\Delta t}
\newcommand{\Ds}{\Delta s}
\newcommand{\V}{\mathbf{V}}
$

In [ ]:
# Commented out IPython magic to ensure Python compatibility.
%pip install -q https://samyzaf.com/fdmtools.zip
from fdmtools.bsm import *

In [ ]:
#@title Black-Scholes-Merton Equation Video
#@markdown  Motivation Video
YouTubeVideo("A5w-dEgIU1M", width=640, height=360)

* The implementation is the straightforward modification of
  Antoni Smolski code.

* I highly recommend reading his Blog!  
  https://antonismolski.medium.com/implementation-of-black-scholes-formula-using-finite-difference-method-with-code-965fd0539808  

* Another useful source:  
https://diter.medium.com/option-pricing-using-the-black-scholes-model-without-the-formula-e09235f75fb7

* The **FDM** solution is based on the following obsevations.
* First, the **BSM** equation can be written as   
  $$
  \large
  -v_t = rv - rsv_s - \frac{1}{2}\sigma^2 s^2 v_{ss}
  $$

* Let $\V$ be an **FDM** discrete grid model for the solution
  $v(s,t)$ of the **BSM** equation over a rectangular
  domain $[0,S]{\times}[0,T]$, where $S$ is the maximal
  asset price $s$, and $T$ is the maximal
  option maturation time $t$.

* The price interval $[0,S]$ is divided to discrete
  nodes $s_i = i\Ds$.
  The time interval $[0,T]$ is divided to time
  nodes $t_k = k\Dt$.
  The grid $\V$ models the solution $v(s,t)$ by
  $\V[i,k] = v(s_i,t_k)$.

* The FDM derivatives we need are as follows:

\begin{align*}
\V_s[i,k] &= \frac{\V[i+1,k] - \V[i-1,k]}{2\Ds}
\\[2.0ex]
\V_{ss}[i,k] &= \frac{\V[i+1,k] -2\V[i,k] + \V[i-1,k]}{\Ds^2}
\\[2.0ex]
\V_t[i,k] &= \frac{\V[i,k+1] - \V[i,k]}{\Dt}
\end{align*}

* From the first observation it follows that

$$
-\frac{\V[i,k+1] - \V[i,k]}{\Dt}
=
r \V[i,k] - rs\V_s[i,k] - \frac{1}{2}\sigma^2 s^2 \V_{ss}[i,k]
$$

* In our discrete model, $s=i\Ds$, and therefore  
  $$
  -\frac{\V[i,k+1] - \V[i,k]}{\Dt}
  =
  r \V[i,k] - r (i\Ds) \V_s[i,k] - \frac{1}{2}\sigma^2 (i\Ds)^2 \V_{ss}[i,k]
  $$

* From which we derive our recursive solution to the **BSM**
  equation:  
  $$
  \V[i,k+1]
  =
  \V[i,k] -
  \Dt \left(r \V[i,k] - r (i\Ds) \V_s[i,k] - \frac{1}{2}\sigma^2 (i\Ds)^2 \V_{ss}[i,k]\right)
  $$

* Many texts use the following abbreviation
$$
\gamma = r \V[i,k] - r (i\Ds) \V_s[i,k] + \frac{1}{2}\sigma^2 (i\Ds)^2 \V_{ss}[i,k]
$$
So we get a shorter form of this solution
$$
\V[i,k+1] = \V[i,k] + \gamma\Dt
$$

* If we implement $\V$ as a Python Numpy array, we can write
  a simple Python procedure for implementing this solution
  in Python.

In [ ]:
# sigma = the volatility of the stock (standard deviation)
#         (the degree of variation of a trading price series over time)
# r = risk-free interest rate
# K = Strike price
# Ns = Number of asset price steps
# Nt = Number of time steps (Nt=0 means choose optimal number)

def Solve(sigma, r, K, T, Ns, Nt=0):
    S = 3*K                             # Max asset price
    ds = S/Ns                           # asset price step
    if Nt==0:                           # Automatic optimal selection of time step
        dt = 0.9 / (sigma**2 * Ns**2)   # Stable optimal value of dt
        Nt = int(T/dt) + 1              # Number of time steps
    dt = T/Nt                           # Ensure that T is an integer number of time steps away
    V = np.zeros((Ns+1, Nt+1))          # Option Value Array (V FDM grid)

    for i in range(Ns+1):
        V[i,0] = max(i*ds - K, 0)       # Initial condition at time t=0

    for k in range(0, Nt+1):            # Time loop
        V[0,k] = 0                      # Boundary condition at s=0
        V[Ns,k] = S - K*exp(-r*k*dt)    # Boundary condition at s=S (Smax)

    for k in range(0, Nt):      # Time loop
        # And now ... Nobel prize winning diff equation ;)
        for i in range(1, Ns):  # Asset loop
            Vs = (V[i+1,k] - V[i-1,k]) / (2*ds)               # Vs[i,k] as central difference
            Vss = (V[i+1,k] - 2*V[i,k] + V[i-1,k]) / (ds**2)  # Vss[i,k] as central difference
            gamma = r*V[i,k] - r*(i*ds) * Vs - 0.5 * sigma**2 * (i*ds)**2 * Vss
            V[i,k+1] = V[i,k] - gamma * dt

    asset_steps = ds * np.arange(0, Ns+1)   # Asset price steps from 0 to S (i*ds, i=0,1,..,Ns)
    time_steps = dt * np.arange(0, Nt+1)    # Time steps from 0 to T (k*dt, k=0,1,..,Nt)
    rounded_asset_steps = np.round(asset_steps, decimals=3)
    rounded_time_steps = np.round(time_steps, decimals=4)
    data = pd.DataFrame(np.transpose(V), index=rounded_time_steps, columns=rounded_asset_steps).round(3)
    data.attrs = dict(S=S, T=T, K=K, ds=ds, dt=dt, r=r, sigma=sigma, Ns=Ns, Nt=Nt)
    return data  # Output array as pandas data frame object

* The boundary conditions are explained in our
  [project booklet](https://samyzaf.com/fdm/fdm.pdf).

# A Simple Example
* In the following example we solve the **BSM** equation
  for a stock call option
  with the following parameters.

* We would like to find the value of an option on a stock
  which is currently priced **s=50**, for an equal
  strike price **K=50**, for maturation time **t=2** (2 years).

* In other words: the current price stock is 50$, and we would
  like to buy an option to purchase it again in 2 years for
  the same price (K=50).
  
* This option should cost us some money. The question is how much?

In [ ]:
K = 50            # Option strike price
S = 3*K           # Maximum asset price
T = 3             # Maximum time (in year units)
Ns = 100          # number of asset price steps
Nt = 0            # Let Solve choose an optimal number of time steps
sigma = 0.4       # Asset price sensitivity (aka volatility)
r = 0.05          # Interest rate

data = Solve(sigma, r, K, T, Ns, Nt)   # We solve the BSM equation!

* **data** is a [Pandas](https://pandas.pydata.org/)
  [DataFrame object](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)
  which holds the FDM grid solution in a tabular form.

* This is a very useful information container which is
  easy to query. It supports more than 400 query types!

* Here are the first 5 rows of this table followed
  by the last 5 rows.
  
* The columns (keys) are asset price steps,
  and the lines are time steps.

* The simplest query is the **data** command itself which
  prints a small sample of of the table, which gives us
  a rough impression about the data it holds.

In [ ]:
data

* In the first column (from the left) we see the
  table **Index**. These are our **Time Steps**.
* The first row is called the **table header**.
  These are our **Price Steps**.

* On the right side of this table you will find three
  buttons for interacting with it just as if it was
  an Excel table.
  * The first button displays a full interactive table.
  * The second buttons display several charts, and
     can create code for generating them.
  * The third button is experimental AI code generator for
     extracting information from this table.

* To learn more on how you can interact with the
  **dataframe** object
  read the
  [Pandas package reference guide](https://pandas.pydata.org/docs/user_guide/index.html)

* For example, to print the table index (time steps),
  use the command:

In [ ]:
times = data.index
print(times)

* This is a list of 5335 time steps that are required for
  precision.
  
* To see the **table keys** (asset price steps),
  use the following command:

In [ ]:
prices = data.keys()
print(prices)

* This is a list of 100 price steps with **ds=1.5**.
* Only a small sample is printed: the first 10 items, and
  the last 10 items.
* A Pandas **DataFrame** can be also exported into a
  Microsoft Excel sheet easily!
* If you are a Microsoft Excel user, you may want to convert
  it to an Excel sheet, and then interact with it using
  Microsoft office analysis tools.

In [ ]:
data.to_excel("data.xlsx")

* The resulting Excel file can be viewed
  [from this link](https://samyzaf.com/fdm/data.xlsx)

* But you can download this Excel sheet from here to your
  local disk by the following command.

In [ ]:
file_download("data.xlsx")

* There are 427 methods that you can
  apply on a **DataFrame** object.

* To learn and try more of them check out the
  [DataFrame reference guide](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).

# BSM Heatmap
* Use the **bsm_heatmap** for visualizing the **data**
  table as a heat map.

* This method is part of our
  [**fdmtools package**](https://samyzaf.com/fdmtools.zip)
  and can be viewed by downloading this package.

In [ ]:
bsm_heatmap(data)

# BSM 3D surface representation
* There are more visualization aids available in Python's
  matplotlib package.

* Here is a 3D surface visualization of our **data** table.

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
Y,X = np.meshgrid(data.columns, data.index)
Z = data.values
surf = ax.plot_surface(X, Y, Z, cmap="jet")
ax.set_xlabel('Time')
ax.set_ylabel('Asset Price')
ax.set_zlabel('Option Value')
ax.set_title('Option Value Surface Plot')
fig.colorbar(surf, shrink=0.5, aspect=8, pad=0.1)
plt.show()

# Extracting local and precision information
* The **data** object is quite large: 855 rows × 101 columns.
  That is 86355 entries!

* The heatmap and 3D surface plots are great **infographics**
  tools, but are not adequate for obtaining precise values
  for specific needs.

* In most cases we would like to inspect a small
  subset of information
  around specific values such as our strike price **K** and
  maturation time **t**.

* For example, if our strike price is **K=50** and maturation
  time is **t=60/365** (2 months), here is a small code snippet
  for extracting a small table around these values:

In [ ]:
s = 50  # initial stock price and also our strike price
t = 60/365   # Maturation time (60 days)
ds = data.attrs["ds"]
dt = data.attrs["dt"]
i = int(s/ds)
k = int(t/dt)
print(f"Extracting a small sub-table (10x10) around price s={s} and time t={t}")
df = data.iloc[k-5:k+5, i-5:i+5]
df

* Notice the **Next steps** suggestions at the bottom of
  the table which offers you code for generating plots.
  You can learn more by playing with it.
  
* The **data** object has a special attributes
  dictionary **data.attrs**
  which holds all the parameters: **S, K, T, ds, dt, ...**
  * To access the time step **dt** use:
    **dt = data.attrs["dt"]**.
  * To access the number of price steps **Ns** use:
    **Ns = data.attrs["Ns"]**.
  * Etc ...

* Like in Microsoft Excel, to access **data** we need integer
  indices **i**, **k**.
  Beware that **k** is a row index, and **i** is a column index!

* Therefore we had to convert the price value **s=50**
  to its corresponding column index **i=s/ds**,
  and the time value **t=60/365** to its corresponding
  row index **k=t/dt**.

* To access 10 rows around **i** we use a range expression
  **i-5:i+5**.

* But the first thing we should have noticed is that
  our desired price **s=50** and time **t=60/365=0.16438**
  are **not** in this table!

* This is a typical issue with discrete models, which is
  confronted by a special interpolation procedure for
  computing the in between values.

* Our **fdmtools** package includes an **interpolate** method
  for handling this issue

In [ ]:
s = 50.0
t = 0.16438     # 60 days = 0.16438 of 1 year
v = interpolate(data, s, t)
print(v)

* So the value of a **CALL option** with price **s=50**
  and maturation time **t=2** years with
  a strike price **K=50** is **v=13.148**!
  This answers our initial problem.

* To make sure that your work is not detached from reality,
  it is recommended that you check if you get a similar result
  in an online option calculator (there are many of them
  on the internet).

* Here's is the result that we got from
  **https://www.barchart.com/options/options-calculator**

  <IMG src="https://samyzaf.com/fdm/opt_calc_ex2.jpg" width=640 align="center"/>

* Looks pretty close to what we got with our code: **v=3.4389**.

* We can also reconsider our maturation time to be **t=1.0**.
  We have a method for generating a local table around
  **s=50, t=1.0**.

* For that purpose we need a more general method for generating
  small subsets of our large table.
  For example, to generate a smaller table for price range 40-60
  and time range 0.98-1.02, we can use the following code:

In [ ]:
times = [t for t in data.index if 0.98<=t<=1.02]
prices = [s for s in data.keys() if 40<=s<=60]
df = data.loc[times,prices]
df

* This is still a long table (71 rows, 15 columns).
* We can use our heatmap or 3d surface tools to get
  a visual impression of it.

In [ ]:
bsm_heatmap(df)

In [ ]:
bsm_surface(df)

* We can perform more precise queries that collect data from
  several locations of our table.
* For example, we can include a small sample around t=1
  and a small sample around t=2.

In [ ]:
times = [t for t in data.index if 0.99<=t<=1.01 or 1.99<=t<=2.01]
prices = [s for s in data.keys() if 40<=s<=60]
df = data.loc[times,prices]
df

In [ ]:
bsm_heatmap(df)

In [ ]:
times = [t for t in data.index if 0.995<=t<=1.005]
prices = [s for s in data.keys() if 44<=s<=56]
df = data.loc[times,prices]
bsm_heatmap(df,vlabel=True)